In [21]:
import os
from os import path
import json
import numpy as np
import pandas as pd

In [39]:
path_results = './results/'

acceptance_policy = 'acceptall'
path_acceptance = path.join(path_results, acceptance_policy)

# placement_policies = list(os.listdir(path_acceptance))
placement_policies = ['default', 'greedy', 'best', 'gavel', 'sgavel']
scheduling_policies = ['fifo', 'las']

range_str = '0_500'
test_prefix = f'test_{range_str}_load_'
job_loads = [10, 50]

percentiles = [ 5, 50, 95 ]
# Each test is a combination of placer, scheduler, and load.
# For each of these combinations, we want to display JCT: mean, common percentiles (5%, 50%, 95%), number of swaps, cluster utilization

# This is difficult to show compactly and we might need to split this into 4 tables, perhaps one for each combination of scheduler and load

test_configurations = []
test_means = []
test_percentiles = {perc: [] for perc in percentiles}
test_swaps = []
for load in job_loads:
    for scheduler_name in scheduling_policies:
        for placement_name in placement_policies:        

            # Collect configuration parameters
            test_configurations.append((placement_name, scheduler_name, load))
            
            # Collect JCT summary statistics
            path_to_test_dir = path.join(path_acceptance, placement_name, scheduler_name)
            test_metric_str = 'job_stats'
            filename_test = f'{test_prefix}{load}_{test_metric_str}.json'
            path_test = path.join(path_to_test_dir, filename_test)
            # print(path_test)

            with open(path_test, "r") as fin:
                data_job = json.load(fin)
            # print(data_job)
            job_durations = [v[1] - v[0] for v in data_job.values()]
            test_mean = np.mean(job_durations)
            # print(test_mean)
            test_means.append(test_mean)

            for p in percentiles:
                jct_perc = np.percentile(job_durations, p)
                # print(f'{p} percentile JCT: {jct_perc}')
                test_percentiles[p].append(jct_perc)

            # Collect number of preemptions ("swaps")
            test_metric_str = 'custom_metric'
            filename_test = f'{test_prefix}{load}_{test_metric_str}.json'
            path_test = path.join(path_to_test_dir, filename_test)

            with open(path_test, "r") as fin:
                data_job = json.load(fin)
            num_preemptions = np.sum([v['num_preemptions'] for v in data_job.values()])
            test_swaps.append(num_preemptions)

# print(test_configurations)
# print(test_means)
# print(test_percentiles)

metrics_JCT = pd.DataFrame({'Test Configuration':test_configurations , 'Mean': test_means, '5th percentile': test_percentiles[5], 'Median': test_percentiles[50], '95th Percentile': test_percentiles[95], '# Swaps': test_swaps})

# sample_row = {'Test Configuration': 'Hamburger', 'Mean': 33, '5th percentile': 55, 'Median': 77, '95th Percentile': 99}
#
# metrics_JCT._append(sample_row, ignore_index=True)

display(metrics_JCT)

,Test Configuration,Mean,5th percentile,Median,95th Percentile,# Swaps
0,"(default, fifo, 10)",164227.285429,23126.0,131525.0,312699.0,0
1,"(greedy, fifo, 10)",21441.057884,713.0,4892.0,108311.0,0
2,"(best, fifo, 10)",17465.009980,814.0,5048.0,87890.0,3833
3,"(gavel, fifo, 10)",18830.878244,808.0,5048.0,94538.0,2869
4,"(sgavel, fifo, 10)",18855.429142,827.0,4881.0,95705.0,3512
5,"(default, las, 10)",46413.512974,735.0,7765.0,188770.0,214
6,"(greedy, las, 10)",21441.057884,713.0,4892.0,108311.0,0
7,"(best, las, 10)",17478.782435,808.0,4836.0,87890.0,3450
8,"(gavel, las, 10)",18855.429142,778.0,4990.0,92571.0,2832
9,"(sgavel, las, 10)",18850.638723,774.0,4877.0,94538.0,3531
